**opendatasets**: can be use to download kaggle datasets directly to colab notebook environment.
**datasets**: Here pandas dataframe is converted into datasets object

In [1]:
!pip install opendatasets datasets accelerate rouge_score --quiet
import opendatasets as od
od.download("https://www.kaggle.com/datasets/gondimalladeepesh/nvidia-documentation-question-and-answer-pairs")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.0 MB/s eta 0:00:00
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: sadeepa
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/gondimalladeepesh/nvidia-documentation-question-and-answer-pairs


100%|██████████| 400k/400k [00:00<00:00, 61.1MB/s]

In [2]:
import pandas as pd
from datasets import Dataset
import datasets
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import re # re module is used for text cleaning purposes

device = 'cuda' if torch.cuda.is_available() else 'cpu' # detect the GPU if any, if not use CPU (If mac use mps instead of cuda)

In [3]:
# only importing "question", "answer" columns from dataset
data = pd.read_csv("/content/nvidia-documentation-question-and-answer-pairs/NvidiaDocumentationQandApairs.csv")[["question", "answer"]]
print(data.shape)
data.head()

(7108, 2)


,question,answer
0,What is Hybridizer?,Hybridizer is a compiler from Altimesh that en...
1,How does Hybridizer generate optimized code?,Hybridizer uses decorated symbols to express p...
2,What are some parallelization patterns mention...,The text mentions using parallelization patter...
3,How can you benefit from accelerators without ...,You can benefit from accelerators' compute hor...
4,What is an example of using Hybridizer?,An example in the text demonstrates using Para...


This function is designed to clean the input text by:

Converting all characters to lowercase (text.lower()).

Using the re.sub() function to substitute any character that is not a letter or a number with a space (re.sub('[^A-Za-z0-9]+', ' ', text)).

In [4]:
def clean_text(text):
  text = text.lower()
  text = re.sub('[^A-Za-z0-9]+', ' ', text)
  return text

# The clean_text function is applied to the 'question' and 'answer' columns of the DataFrame
data['question'] = data['question'].apply(clean_text)
data['answer'] = data['answer'].apply(clean_text)
data.head()

,question,answer
0,what is hybridizer,hybridizer is a compiler from altimesh that en...
1,how does hybridizer generate optimized code,hybridizer uses decorated symbols to express p...
2,what are some parallelization patterns mention...,the text mentions using parallelization patter...
3,how can you benefit from accelerators without ...,you can benefit from accelerators compute hors...
4,what is an example of using hybridizer,an example in the text demonstrates using para...


In [5]:
train=data.sample(frac=0.7,random_state=7) # Create training of 70% of the data
test=data.drop(train.index) # Create testing by removing the 70% of the train data which will result in 30%

val=test.sample(frac=0.5,random_state=7) # Create validation of 50% of the testing data
test=test.drop(val.index) # Create testing by removing the 50% of the validation data which will result in 50%

In [6]:
print("Training Shape: ", train.shape)
print("Validation Shape: ", val.shape)
print("Testing Shape: ", test.shape)

Training Shape:  (4976, 2)
Validation Shape:  (1066, 2)
Testing Shape:  (1066, 2)


In [7]:
def get_model_tokenizer(model_name = "google/flan-t5-base"):
  original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.float32)
  tokenizer = AutoTokenizer.from_pretrained(model_name)

  return original_model, tokenizer

In [8]:
model, tokenizer = get_model_tokenizer()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [9]:
rouge_metric = datasets.load_metric("rouge")

<ipython-input-9-960def13b8d7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = datasets.load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(




*   prompt: This is the input text that needs to be tokenized.


*   padding="max_length": This parameter ensures that all sequences are padded to the same length, which is specified by max_length. Padding is usually done by adding special tokens (like zeros) to the sequence to ensure they all have the same length.

*   truncation=True: This parameter ensures that sequences longer than max_length are truncated to the specified maximum length. Truncation helps in preventing excessively long sequences that may exceed the model's capacity.


*   return_tensors="pt": This parameter specifies the type of tensor to return. "pt" indicates that the tokenizer should return PyTorch tensors. If you were using TensorFlow, you might use "tf" instead.

*  max_length=200: This sets the maximum length of the tokenized sequences. Sequences will be either padded up to or truncated down to this length.



*   .input_ids: After tokenizing, the tokenizer function returns a dictionary. It contains input_ids and attention_mask. We need only input_ids. The input_ids key contains the tokenized representation of the input text (as a tensor).











In [10]:
def tokenize_function(example):
    start_prompt = '\n\n'
    end_prompt = '\n\nAnswer: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["question"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt", max_length=200).input_ids
    example['labels'] = tokenizer(example["answer"], padding="max_length", truncation=True, return_tensors="pt",max_length=200).input_ids

    return example

In [11]:
# Convert padas data frame into a dataset
train_data = Dataset.from_pandas(train)
# dataset is tokenizing
train_tokenized_datasets = train_data.map(tokenize_function, batched=True)
# Remove unwanted columns
train_tokenized_datasets = train_tokenized_datasets.remove_columns(['question', 'answer',])


val_data = Dataset.from_pandas(val)
val_tokenized_datasets = val_data.map(tokenize_function, batched=True)
val_tokenized_datasets = val_tokenized_datasets.remove_columns(['question', 'answer',])


test_data = Dataset.from_pandas(test)
test_tokenized_datasets = test_data.map(tokenize_function, batched=True)
test_tokenized_datasets = test_tokenized_datasets.remove_columns(['question', 'answer',])

Map:   0%|          | 0/4976 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [29]:
print(len(train_tokenized_datasets["input_ids"][1]))

200


In [30]:
print(len(train_tokenized_datasets["labels"][1]))

200


Dataset: We have a dataset of 4976 training samples.

BATCH_SIZE = 2:

In this case, the model will process 2 samples at a time before updating the model's weights.

For 4976 samples, there will be 2488 batches per epoch (4976 samples / 2 samples per batch).

First Batch: The model processes the first 2 samples, calculates the loss, and updates the weights.

1. Forward Pass: The model processes 2 samples at a time. It predicts the output for both samples in the batch.
2. Loss Calculation: The loss is calculated for each sample in the batch. Then, the average loss for the batch is computed.
3. Backward Pass: The gradients are calculated based on the average loss, and the model's weights are updated accordingly.

Second Batch: The model processes the next 2 samples, calculates the loss, and updates the weights.

This process continues until all 4976 samples are processed.


In [12]:
EPOCHS = 1
LR = 1e-4
BATCH_SIZE = 2

# model checkpoints are saved in this location
# If training process interrupt saved model versions are important
# save_total_limit = 2 defines how many versions to save
training_path = "./training_path_nvidia_chatbot"

training_args = TrainingArguments(
    output_dir = training_path,
    overwrite_output_dir = True,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size = BATCH_SIZE,
    learning_rate = LR,
    num_train_epochs = EPOCHS,
    evaluation_strategy = "epoch",
    save_total_limit = 2
    )

trainer= Trainer(
    model = model,
    args = training_args,
    train_dataset = train_tokenized_datasets,
    eval_dataset = val_tokenized_datasets,
)

trainer.train()

# final trained model saved to this location
model_path = "./nvidia-chatbot-final-model"

trainer.model.save_pretrained(model_path)
# Actually there is no need to save takenizer
# Because we are using original tokenizer everytime
tokenizer.save_pretrained(model_path)

Epoch,Training Loss,Validation Loss
1,0.487900,0.438704


('./nvidia-chatbot-final-model/tokenizer_config.json',
 './nvidia-chatbot-final-model/special_tokens_map.json',
 './nvidia-chatbot-final-model/spiece.model',
 './nvidia-chatbot-final-model/added_tokens.json',
 './nvidia-chatbot-final-model/tokenizer.json')

In [13]:
rouge_metric = datasets.load_metric("rouge")
# Evaluate using evaluating dataset
eval_results= trainer.evaluate(
    eval_dataset = test_tokenized_datasets,
                 )

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [14]:
print(eval_results)

{'eval_loss': 0.4375801384449005, 'eval_runtime': 49.8139, 'eval_samples_per_second': 21.4, 'eval_steps_per_second': 10.7, 'epoch': 1.0}


**num_beams=1**

num_beams is a parameter related to the beam search algorithm, which is a technique used in text generation tasks to improve the quality of generated sequences. Beam search explores multiple possible sequences simultaneously and selects the most likely sequence based on their probabilities.

num_beams=1: When num_beams is set to 1, the model uses a simpler approach called greedy search instead of beam search. In greedy search, the model selects the most probable next token at each step without considering alternative sequences. Essentially, it is the fastest but least thorough search method.

Beam Search with Higher num_beams: If num_beams were greater than 1, the model would consider multiple candidate sequences (equal to num_beams) at each step, keeping the most promising ones. This generally leads to better quality output but increases computation time.

**Selecting the First Sequence: [0]** is used to select the first generated sequence from the batch. This is useful when you input a single text and expect a single generated output, but the model still returns the result in a batch format.

In [15]:
# Predic using fine tunied model
test_text = "what is cuda nsight?"
# Importing fine tuned saved model and tokenizer
trained_model, tokenizer = get_model_tokenizer(model_path)
# Tokenizing dataset
tokenized_test_text = tokenizer(test_text,
                              return_tensors='pt')
# Default max token generation equal to 20
# That's why we import GenerationConfig and set max_new_tokens to 200
model_output = trained_model.generate(tokenized_test_text.input_ids,
                                      generation_config=GenerationConfig(max_new_tokens=200, num_beams=1),)[0]
# Decode and remove special tokens
final_output = tokenizer.decode(model_output, skip_special_tokens=True)
print(final_output)

cuda nsight is a gpu based nsight toolkit that provides a comprehensive overview of the cuda gpu architecture and provides a comprehensive overview of the gpu architecture and its features 


In [22]:
original_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.float32).to(device)
original_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

In [23]:
# Predict from original model
test_text = "what is cuda nsight?"

tokenized_test_text = tokenizer(test_text,
                              return_tensors='pt').to(device)
model_output = original_model.generate(tokenized_test_text.input_ids,
                                      generation_config=GenerationConfig(max_new_tokens=200, num_beams=1),)[0]
final_output = tokenizer.decode(model_output, skip_special_tokens=True)
print(final_output)

a snare
